<a href="https://colab.research.google.com/github/gnloop/MFA-Universal-Notebook/blob/slicer/utau_conversion/Japanese-Conversion-BAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **[Montreal Forced Aligner](https://montreal-forced-aligner.readthedocs.io/)**
_Command line utility for forced alignment using Kaldi_

\
____
\

## _Tool to automatically convert hiragana voicebanks into datasets for DiffSinger_

\
____


# Huge thanks to PixPrucer and HAI-D for basically making every part of this notebook. I just updated things around basically ⛹

In [ ]:
#@title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Install Condacolab
#@markdown The session will crash and restart, that's normal!
!pip install -q condacolab
import condacolab
condacolab.install()
from IPython.display import clear_output
clear_output()
print("All done, please wait for the session to restart!")

In [ ]:
#@title Install MFA
!conda install -c conda-forge montreal-forced-aligner spacy sudachipy sudachidict-core
!pip install speechbrain
!git clone https://github.com/gnloop/MFA-Universal-Notebook
!mfa model download acoustic japanese_mfa
!cp /content/MFA-Universal-Notebook/utau_conversion/jpn_utau.dict /root/Documents/MFA/pretrained_models/dictionary
from IPython.display import clear_output
clear_output()
print("All done!")

In [ ]:
#@title Unzip VB and generate & edit transcriptions
#@markdown Unzip your voicebank, make sure it is a zip file with ONLY .wav files encoded in hiragana. It is HIGHLY recommended to only work on one pitch at a time. Please note that anything that isn't standard hiragana might not work.

file_location = '/content/drive/MyDrive/wav.zip' #@param {type:"string"}

!7z x "$file_location" -o/content/db
import os
import re

def extract_hiragana(wav_filename):
    """Extracts hiragana characters from a filename."""
    hiragana_only = re.sub(r"[^ぁ-んァ-ン]", "", wav_filename)
    return hiragana_only

def generate_and_modify_txt_files(input_folder, output_folder):
    """Generates and modifies .txt files containing hiragana from .wav filenames."""
    for filename in os.listdir(input_folder):
        if filename.endswith(".wav"):
            wav_filepath = os.path.join(input_folder, filename)
            hiragana_text = extract_hiragana(filename)

            txt_filename = os.path.splitext(filename)[0] + ".txt"
            txt_filepath = os.path.join(output_folder, txt_filename)

            with open(txt_filepath, "w", encoding="utf-8") as txt_file:
                # Directly write the modified content with spaces:
                modified_content = re.sub(r"([ぁ-んー][ゃゅょっゎァィゥェォぁぃぅぇぉ]?)", r"\1 ", hiragana_text)
                modified_content = re.sub(r" +", " ", modified_content)  # Remove extra spaces
                txt_file.write(modified_content)

# Set the input and output folders
input_folder = "/content/db"
output_folder = "/content/db"  # Save .txt files in the same folder as .wav files

# Generate and modify the .txt files
generate_and_modify_txt_files(input_folder, output_folder)
from IPython.display import clear_output
clear_output()
print("Wavs extracted and transcriptions generated & edited automatically in db folder")

In [ ]:
#@title Start aligning!
!mfa align /content/db jpn_utau japanese_mfa /content/alignment --beam 400
from IPython.display import clear_output
clear_output()
print("All done! Check the 'alignment' folder for .TextGrid files")

In [ ]:
#@title Install TextGrid to LAB converter
!pip install mytextgrid
!mv /content/MFA-Universal-Notebook/text2lab_test.py /content/alignment/
from IPython.display import clear_output
clear_output()
print("All done!")

In [ ]:
#@title Convert TextGrid to LAB
!python -X utf8 /content/alignment/text2lab_test.py -c /content/MFA-Universal-Notebook/converters/converter_JP.txt
from IPython.display import clear_output
clear_output()
print("Your .lab files should now be under the 'alignment' folder!")


In [ ]:
#@title Zip output
# @markdown Enter the path where you want your labels saved (no / at the end)
zip_path = '/content/drive/MyDrive/MFA_output' #@param {type:"string"}
from IPython.display import display, HTML, Javascript
import os

# Create the directory if it doesn't exist
os.makedirs(zip_path, exist_ok=True)

# Zip labels
!zip -j {zip_path}/labels.zip /content/alignment/*.lab

from IPython.display import clear_output
clear_output()
print("You can now download your labels in the labels.zip file and wavs in the sliced_wavs.zip file!")